In [1]:
!find / -name "*.zip"

find: ‘/proc/57/task/57/net’: Invalid argument
find: ‘/proc/57/net’: Invalid argument
/usr/lib/R/site-library/vroom/extdata/mtcars-multi-cyl.zip
/usr/lib/R/site-library/vroom/extdata/mtcars.csv.zip
/usr/lib/R/site-library/data.table/tests/russellCRLF.zip
/usr/lib/R/site-library/data.table/tests/multi-file.zip
/usr/lib/R/site-library/zip/example.zip
/usr/lib/R/site-library/readr/extdata/mtcars.csv.zip
/usr/lib/jvm/java-11-openjdk-amd64/lib/src.zip
/usr/local/lib/python3.10/dist-packages/tensorboard/webfiles.zip
/usr/local/lib/python3.10/dist-packages/pygame/docs/generated/_static/legacy_logos.zip
/usr/local/lib/python3.10/dist-packages/geopandas/datasets/nybb_16a.zip
/usr/local/lib/python3.10/dist-packages/pooch/tests/data/tiny-data.zip
/usr/local/lib/python3.10/dist-packages/pooch/tests/data/store.zip


In [2]:
filepath = "/usr/lib/R/site-library/zip/example.zip"

In [3]:
import zipfile

In [4]:
with zipfile.ZipFile(filepath,"r") as myzip:
    for info in myzip.infolist():
        print(info.filename)

example/
example/dir/
example/dir/file2
example/file1


In [5]:
with zipfile.ZipFile(filepath,"r") as myzip:
    with myzip.open("example/file1") as myfile:
        print(myfile.read())

b'file1\n'


In [6]:
def dump(body):
    print("         0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F")
    ascii = ""
    i = 0
    l = 0
    print(f"{l:08X}:", end="")
    for byte in body:
        print(f"{byte:02X} ", end="")
        if byte >= 0x20 and byte <= 0x7e:
            ascii += chr(byte)
        else:
            ascii += '.'
        i += 1
        if i == 16:
            print(" ",ascii)
            ascii = ""
            i = 0
            l += 16
            print(f"{l:08X}:", end="")
    if i != 0:
        print("   "*(16-i),end="")
        print(" ",ascii)

def dump2(filepath):
    with open(filepath, "rb") as f:
        body = f.read()
        dump(body)

In [7]:
dump2(filepath)

         0  1  2  3  4  5  6  7  8  9  A  B  C  D  E  F
00000000:50 4B 03 04 00 00 00 08 00 00 07 AF AA 56 00 00   PK...........V..
00000010:00 00 00 00 00 00 00 00 00 00 08 00 00 00 65 78   ..............ex
00000020:61 6D 70 6C 65 2F 50 4B 03 04 00 00 00 08 00 00   ample/PK........
00000030:07 AF AA 56 00 00 00 00 00 00 00 00 00 00 00 00   ...V............
00000040:0C 00 00 00 65 78 61 6D 70 6C 65 2F 64 69 72 2F   ....example/dir/
00000050:50 4B 03 04 14 00 08 08 08 00 07 AF AA 56 00 00   PK...........V..
00000060:00 00 00 00 00 00 00 00 00 00 11 00 00 00 65 78   ..............ex
00000070:61 6D 70 6C 65 2F 64 69 72 2F 66 69 6C 65 32 01   ample/dir/file2.
00000080:06 00 F9 FF 66 69 6C 65 32 0A 50 4B 07 08 C7 A4   ....file2.PK....
00000090:04 C9 0B 00 00 00 06 00 00 00 50 4B 03 04 14 00   ..........PK....
000000A0:08 08 08 00 07 AF AA 56 00 00 00 00 00 00 00 00   .......V........
000000B0:00 00 00 00 0D 00 00 00 65 78 61 6D 70 6C 65 2F   ........example/
000000C0:66 69 6C 65 31 01 06 00

In [8]:
with open(filepath,"rb") as f:
    body = f.read()

    #Find End of Central Directory
    offset_eocd = body.rfind(b"\x50\x4b\x05\x06")
    print(f"{offset_eocd:08X}")


000001CA


In [9]:
import struct

with open(filepath,"rb") as f:
    body = f.read()

    offset_eocd = body.rfind(b"\x50\x4b\x05\x06")
    body_eocd = struct.unpack("<4s4H2LH",body[offset_eocd:offset_eocd+22])
    print(body_eocd)


(b'PK\x05\x06', 0, 0, 4, 4, 234, 224, 0)


In [10]:
import struct

with open(filepath,"rb") as f:
    body = f.read()

    offset_eocd = body.rfind(b"\x50\x4b\x05\x06")
    body_eocd = struct.unpack("<4s4H2LH",body[offset_eocd:offset_eocd+22])

    offset = body_eocd[6]
    offset_cd = [0 for i in range(body_eocd[4])]
    for i in range(body_eocd[4]):
        offset_cd[i] = offset
        (n,m,k) = struct.unpack("<3H",body[offset+28:offset+34])
        print(f"{offset:08X}:{n},{m},{k}")
        offset += 46 + n + m + k




000000E0:8,0,0
00000116:12,0,0
00000150:17,0,0
0000018F:13,0,0


In [11]:
import struct

with open(filepath,"rb") as f:
    body = f.read()

    offset_eocd = body.rfind(b"\x50\x4b\x05\x06")
    body_eocd = struct.unpack("<4s4H2LH",body[offset_eocd:offset_eocd+22])

    offset = body_eocd[6]
    offset_cd = [0 for i in range(body_eocd[4])]
    offset_lf = [0 for i in range(body_eocd[4])]
    for i in range(body_eocd[4]):
        offset_cd[i] = offset
        (n,m,k) = struct.unpack("<3H",body[offset+28:offset+34])
        offset_lf[i] = struct.unpack("<L",body[offset+42:offset+46])[0]
        print(f"{offset_lf[i]:08X}")
        offset += 46 + n + m + k




00000000
00000026
00000050
0000009A


In [12]:
import struct

with open(filepath,"rb") as f:
    body = f.read()

    offset_eocd = body.rfind(b"\x50\x4b\x05\x06")
    body_eocd = struct.unpack("<4s4H2LH",body[offset_eocd:offset_eocd+22])

    offset = body_eocd[6]
    offset_cd = [0 for i in range(body_eocd[4])]
    offset_lf = [0 for i in range(body_eocd[4])]
    for i in range(body_eocd[4]):
        offset_cd[i] = offset
        (n,m,k) = struct.unpack("<3H",body[offset+28:offset+34])
        offset_lf[i] = struct.unpack("<L",body[offset+42:offset+46])[0]
        header_ld = struct.unpack("<4s5H3L2H",body[offset_lf[i]:offset_lf[i]+30])
        (n2,m2) = header_ld[9:11]
        filename = body[offset_lf[i]+30:offset_lf[i]+30+n2].decode()
        print(f"{offset_lf[i]:08X}:{filename},{header_ld[3]}")
        offset += 46 + n + m + k




00000000:example/,0
00000026:example/dir/,0
00000050:example/dir/file2,8
0000009A:example/file1,8


In [13]:
import struct
import zlib

with open(filepath,"rb") as f:
    body = f.read()

    offset_eocd = body.rfind(b"\x50\x4b\x05\x06")
    body_eocd = struct.unpack("<4s4H2LH",body[offset_eocd:offset_eocd+22])

    offset = body_eocd[6]
    offset_cd = [0 for i in range(body_eocd[4])]
    offset_lf = [0 for i in range(body_eocd[4])]
    for i in range(body_eocd[4]):
        offset_cd[i] = offset
        (n,m,k) = struct.unpack("<3H",body[offset+28:offset+34])
        offset_lf[i] = struct.unpack("<L",body[offset+42:offset+46])[0]
        header_ld = struct.unpack("<4s5H3L2H",body[offset_lf[i]:offset_lf[i]+30])
        (n2,m2) = header_ld[9:11]
        filename = body[offset_lf[i]+30:offset_lf[i]+30+n2].decode()
        print(f"{offset_lf[i]:08X}:{filename},{header_ld[3]}")
        if header_ld[3] == 8:
            # -15 for the window buffer will make it ignore headers/footers
            print(zlib.decompress(body[offset_lf[i]+30+n2+m2:],-15))
        offset += 46 + n + m + k




00000000:example/,0
00000026:example/dir/,0
00000050:example/dir/file2,8
b'file2\n'
0000009A:example/file1,8
b'file1\n'


In [14]:
def zip_recovery(body):
    offset_lf = 0
    while True:
        offset_lf = body.find(b"\x50\x4b\03\04",offset_lf)
        if offset_lf == -1:
            break
        header_lf = struct.unpack("<4s5H3L2H",body[offset_lf:offset_lf+30])
        (n,m) = header_lf[9:11]
        filename = body[offset_lf+30:offset_lf+30+n].decode()
        print(f"{offset_lf:08X}:{filename},{header_lf[3]}")
        if header_lf[3] == 8:
            # -15 for the window buffer will make it ignore headers/footers
            print(zlib.decompress(body[offset_lf+30+n+m:],-15))
        offset_lf += 30 + n + m


In [15]:
zip_recovery(body[:0x100])

00000000:example/,0
00000026:example/dir/,0
00000050:example/dir/file2,8
b'file2\n'
0000009A:example/file1,8
b'file1\n'


In [16]:
zip_recovery(body[50:0x100])


0000001E:example/dir/file2,8
b'file2\n'
00000068:example/file1,8
b'file1\n'
